In [ ]:
import os
import re
import sys
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
import matplotlib.pyplot as plt

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

cmap_name = 'viridis'
verbose = True

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\IEEE39 w/ Stoch. Load'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
project = app.GetActiveProject()
if project is None:
    raise Exception('Cannot get active project')
print('Successfully obtained active project.')

In [ ]:
project_folders = {}
for folder_name in ('study',):
    project_folders[folder_name] = app.GetProjectFolder(folder_name)
    if project_folders[folder_name] is None:
        raise Exception(f'No folder "{folder_name}" present')
    print(f'Successfully obtained folder "{folder_name}".')

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
lines = app.GetCalcRelevantObjects('*.ElmLne')
buses = app.GetCalcRelevantObjects('*.ElmTerm')
loads = app.GetCalcRelevantObjects('*.ElmLod')
n_generators, n_lines, n_buses, n_loads = len(generators), len(lines), len(buses), len(loads)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')

In [ ]:
for line in lines:
    vrating = line.typ_id.uline
    print(f'{line.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
for bus in buses:
    vrating = bus.uknom
    print(f'{bus.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
P_load = {}
Q_load = {}
for load in loads:
    name = load.loc_name
    P_load[name] = load.plini
    Q_load[name] = load.qlini
    if verbose: print(f'{name}: P = {P_load[name]:7.2f} MW, Q = {Q_load[name]:6.2f} MVA')

In [ ]:
P_gen = {}
S_gen = {}
H_gen = {}

default_H = {
    'G 01': 5.0,
    'G 02': 4.33,
    'G 03': 4.47,
    'G 04': 3.57,
    'G 05': 4.33,
    'G 06': 4.35,
    'G 07': 3.77,
    'G 08': 3.47,
    'G 09': 3.45,
    'G 10': 4.20
}
coeff = 1
H = {k: coeff*v for k,v in default_H.items()}
output_file = f'IEEE39_H_{coeff:g}x.npz'
for generator in generators:
    #i = get_ID(generator)
    name = generator.loc_name
    generator_type = generator.typ_id
    generator_type.h = H[name]
    P_gen[name] = generator.pgini
    S_gen[name] = generator_type.sgn
    H_gen[name] = generator_type.h
    print(f'{name}: P = {P_gen[name]:4.0f} MW, S = {S_gen[name]:5.0f} MVA, inertia = {H_gen[name]:5.2f} s.')

In [ ]:
areas_map = {
    1: ['G 02', 'G 03', 'G 10'],
    2: ['G 04', 'G 05', 'G 06', 'G 07'],
    3: ['G 08', 'G 09'],
    4: ['G 01']
}
H_area = {}   # inertia
E_area = {}   # energy
M_area = {}   # momentum
for area_id,generator_names in areas_map.items():
    num, den = 0,0
    for generator_name in generator_names:
        num += S_gen[generator_name] * H_gen[generator_name]
        den += S_gen[generator_name]
    H_area[area_id] = num / den 
    E_area[area_id] = num * 1e-3
    M_area[area_id] = 2 * num * 1e-3 / 60
print('Area inertias:  [{}] s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', H_area.values())))))
print('Area energies:  [{}] GW s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', E_area.values())))))
print('Area momentums: [{}] GW s^2.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', M_area.values())))))

## Transient stability analysis

In [ ]:
study_case_name = '5. Transient Stability'
if '.IntCase' not in study_case_name and False:
    study_case_name += '.IntCase'
study_case = project_folders['study'].GetContents(study_case_name)[0]
err = study_case.Activate() # don't know why this returns 1
# if err:
#     raise Exception(f'Cannot activate study case {study_case_name}')
print(f'Successfully activated study case {study_case_name}.')

Objects that will be used in the following:

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
loads = app.GetCalcRelevantObjects('*.ElmLod')
buses = app.GetCalcRelevantObjects('*.ElmTerm')

Find the load that should be stochastic:

In [ ]:
stochastic_load_name = 'Load 03'
found = False
for load in loads:
    if load.loc_name == stochastic_load_name:
        stochastic_load = load
        found = True
        print(f'Found load named {stochastic_load_name}.')
        break
if not found:
    raise Exception(f'Cannot find load named {stochastic_load_name}')

composite_model_name = 'Stochastic Load'
found = False
for composite_model in app.GetCalcRelevantObjects('*.ElmComp'):
    if composite_model.loc_name == composite_model_name:
        stochastic_load_model = composite_model
        found = True
        print(f'Found composite model named {composite_model_name}.')
        break
if not found:
    raise Exception(f'Cannot find composite model named {composite_model_name}')

for slot,net_element in zip(stochastic_load_model.pblk, stochastic_load_model.pelm):
    if slot.loc_name == 'load slot':
        net_element = stochastic_load
        print(f'Set {stochastic_load_name} as stochastic load.')

Find the path of the file containing the dynamics of the stochastic load

In [ ]:
measurement_file_obj = app.GetCalcRelevantObjects('*.ElmFile')[0]
stochastic_load_filename = measurement_file_obj.f_name
print(f'The stochastic load file is {stochastic_load_filename}.')

Write the file with the stochastic load

In [ ]:
frand = 100     # [Hz]
dt = 1 / frand  # [s]
tend = 10 * 60  # [s]
N = int(tend / dt) + 1
seed = 100
stddev = 20     # [MW]
tau = 2         # [s]
P0 = stochastic_load.plini
Q0 = stochastic_load.qlini
rs = RandomState(MT19937(SeedSequence(seed)))
ou = OU(dt, P0, stddev, tau, N, rs)
tPQ = np.zeros((N,3))
tPQ[:,0] = np.linspace(0, tend, N)
tPQ[:,1] = ou
tPQ[:,2] = Q0
with open(stochastic_load_filename, 'w') as fid:
    fid.write('2\n\n')
    for row in tPQ:
        fid.write(f'{row[0]:.6f}\t{row[1]:.2f}\t{row[2]:.2f}\n\n')

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(8,4))
ax[0].plot(tPQ[:,0], tPQ[:,1], color=[.6,.6,.6], lw=1)
ax[0].plot([0, tend], P0 + np.zeros(2), 'b--', lw=2)
ax[1].plot(tPQ[:,0], tPQ[:,2], 'r', lw=1)
for a in ax:
    a.grid(which='major', axis='both', lw=0.5, ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[1].set_xlabel('Time [s]')
ax[0].set_ylabel('P [MW]')
ax[1].set_ylabel('Q [MVAR]')
ax[0].set_xlim([0,tend])
fig.tight_layout()

In [ ]:
monitored_variables = {
    '*.ElmSym':  ['s:xspeed', 'c:fi'],
    '*.ElmLod':  ['m:Psum:bus1', 'm:Qsum:bus1'],
    '*.ElmTerm': ['m:u', 'm:ur', 'm:ui', 'm:u1', 'm:u1r', 'm:u1i', 'm:fe'],
    '*.ElmLne':  ['m:P:bus1', 'm:Q:bus1', 'm:Psum:bus1', 'm:Qsum:bus1', 'm:Psum:bus2', 'm:Qsum:bus2']
}
# the results of the transient simulation will be stored in this variable
res = app.GetFromStudyCase('*.ElmRes')
for elements,var_names in monitored_variables.items():
    for element in app.GetCalcRelevantObjects(elements):
        for var_name in var_names:
            res.AddVariable(element, var_name)

In [ ]:
inc = app.GetFromStudyCase('ComInc')
inc.iopt_sim = 'rms'
inc.iopt_coiref = 2
inc.tstart = 0
inc.dtgrd = dt * 1e3
err = inc.Execute()
if err:
    raise Exception('Cannot compute initial condition')
print('Successfully computed initial condition.')

In [ ]:
%%timeit -n 1 -r 1
sim = app.GetFromStudyCase('ComSim')
sim.tstop = tend
err = sim.Execute()
if err:
    raise Exception('Cannot run transient simulation')
print('Successfully run transient simulation.')
res.Load()

### Get the data

In [ ]:
# we find the buses and the lines in this way (and not with a list comprehension) so they have
# the same order in data_buses and data_lines as in bus_IDs and line_IDs

bus_IDs = (3, 14, 17, 39)
data_buses = []
for bus_ID in bus_IDs:
    for bus in buses:
        if get_ID(bus) == bus_ID:
            data_buses.append(bus)
            break

line_IDs = ((3,4), (14,15), (16,17), (1,39))
data_lines = []
for line_ID in line_IDs:
    for line in lines:
        if get_line_bus_IDs(line) == line_ID:
            data_lines.append(line)
            break

sampling_rate = 10. # [Hz]
dtsim = get_simulation_dt(res)
dec = int((1 / sampling_rate) // dtsim)
print(f'Decimation: {dec}')

sys.stdout.write('Reading time... ')
sys.stdout.flush()
time = get_simulation_time(res, decimation=dec)
sys.stdout.write('done.\n')

sys.stdout.write('Reading generators omega... ')
sys.stdout.flush()
omega = get_simulation_variables(res, 's:xspeed', elements=generators, decimation=dec)
omega_norm = normalize(omega)
sys.stdout.write('done.\n')

sys.stdout.write('Reading generators delta... ')
sys.stdout.flush()
delta = get_simulation_variables(res, 'c:fi', elements=generators, decimation=dec)
sys.stdout.write('done.\n')

sys.stdout.write('Reading electrical frequencies... ')
sys.stdout.flush()
F = get_simulation_variables(res, 'm:fe', elements=data_buses, decimation=dec)
F_norm = normalize(F)
sys.stdout.write('done.\n')

sys.stdout.write('Reading V... ')
sys.stdout.flush()
V = get_simulation_variables(res, 'm:u', elements=data_buses, decimation=dec)
V_norm = normalize(V)
sys.stdout.write('done.\n')

sys.stdout.write('Reading Vd... ')
sys.stdout.flush()
Vd_uncorr = get_simulation_variables(res, 'm:ur', elements=data_buses, decimation=dec)
sys.stdout.write('done.\n')

sys.stdout.write('Reading Vq... ')
sys.stdout.flush()
Vq_uncorr = get_simulation_variables(res, 'm:ui', elements=data_buses, decimation=dec)
sys.stdout.write('done.\n')

sys.stdout.write('Reading Pe... ')
sys.stdout.flush()
Pe = get_simulation_variables(res, 'm:P:bus1', elements=data_lines, decimation=dec)
Pe_norm = normalize(Pe)
sys.stdout.write('done.\n')

sys.stdout.write('Reading Pq... ')
sys.stdout.flush()
Qe = get_simulation_variables(res, 'm:Q:bus1', elements=data_lines, decimation=dec)
Qe_norm = normalize(Qe)
sys.stdout.write('done.\n')

res.Release()

### Compute the corrected values of Vd and Vq

In [ ]:
Vd, Vq = correct_Vd_Vq(Vd_uncorr, Vq_uncorr, delta[:,0])
Vd_norm = normalize(Vd)
Vq_norm = normalize(Vq)

### Save the data

In [ ]:
data = {
    'H_gen': H_gen, 'H_area': H_area, 'E_area': E_area, 'M_area': M_area,
    'time': time, 'F': F, 'F_norm': F_norm, 'V': V, 'V_norm': V_norm,
    'Vd': Vd, 'Vd_norm': Vd_norm, 'Vq': Vq, 'Vq_norm': Vq_norm,
    'Pe': Pe, 'Pe_norm': Pe_norm, 'Qe': Qe, 'Qe_norm': Qe_norm,
    'omega': omega, 'omega_norm': omega_norm, 'Vd_uncorr': Vd_uncorr,
    'Vq_uncorr': Vq_uncorr, 'delta': delta, 'tPQ': tPQ
}
np.savez_compressed(output_file, **data)

### Plot the (normalized) frequency at the buses of interest

In [ ]:
cmap = plt.get_cmap(cmap_name, len(bus_IDs))
fig,ax = plt.subplots(1, 1, figsize=(8,3))
for i,bus in enumerate(data_buses):
    ax.plot(time, F_norm[:,i], color=cmap(i), lw=1, label=bus.loc_name)
ax.legend(loc='upper right')
ax.set_ylim([-4,4])
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Norm. frequency')
ax.grid(which='major', axis='both', lw=0.5, ls=':')
fig.tight_layout()

### Plot the uncorrected voltages at the buses of interest

In [ ]:
cmap = plt.get_cmap(cmap_name, len(bus_IDs))
fig,ax = plt.subplots(2, 1, figsize=(8,5))
for i,bus in enumerate(data_buses):
    ax[0].plot(time, Vd_uncorr[:,i], color=cmap(i), lw=1, label=bus.loc_name)
    ax[1].plot(time, Vq_uncorr[:,i], color=cmap(i), lw=1)
ax[0].legend(loc='upper right')
for a in ax:
    a.set_ylim([-1.2, 1.2])
    a.grid(which='major', axis='both', lw=0.5, ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('Uncorrected Vd')
ax[1].set_ylabel('Uncorrected Vq')
fig.tight_layout()

### Plot the (normalized) corrected voltages at the buses of interest

In [ ]:
cmap = plt.get_cmap(cmap_name, len(bus_IDs))
fig,ax = plt.subplots(2, 1, figsize=(8,5))
for i,bus in enumerate(data_buses):
    ax[0].plot(time, Vd_norm[:,i], color=cmap(i), lw=1, label=bus.loc_name)
    ax[1].plot(time, Vq_norm[:,i], color=cmap(i), lw=1)
ax[0].legend(loc='upper right')
for a in ax:
    a.set_ylim([-4,4])
    a.grid(which='major', axis='both', lw=0.5, ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('Norm. Vd')
ax[1].set_ylabel('Norm. Vq')
fig.tight_layout()

### Plot the active and reactive powers at the buses of interest

In [ ]:
cmap = plt.get_cmap(cmap_name, len(line_IDs))
fig,ax = plt.subplots(2, 1, figsize=(8,5))
for i,bus in enumerate(data_buses):
    ax[0].plot(time, Pe[:,i], color=cmap(i), lw=1, label=bus.loc_name)
    ax[1].plot(time, Qe[:,i], color=cmap(i), lw=1)
ax[0].legend(loc='upper right')
for a in ax:
    a.grid(which='major', axis='both', lw=0.5, ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('Norm. Pe')
ax[1].set_ylabel('Norm. Qe')
fig.tight_layout()

### Plot the (normalized) active and reactive powers at the buses of interest

In [ ]:
cmap = plt.get_cmap(cmap_name, len(line_IDs))
fig,ax = plt.subplots(2, 1, figsize=(8,5))
for i,bus in enumerate(data_buses):
    ax[0].plot(time, Pe_norm[:,i], color=cmap(i), lw=1, label=bus.loc_name)
    ax[1].plot(time, Qe_norm[:,i], color=cmap(i), lw=1)
ax[0].legend(loc='upper right')
for a in ax:
    a.set_ylim([-4,4])
    a.grid(which='major', axis='both', lw=0.5, ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('Norm. Pe')
ax[1].set_ylabel('Norm. Qe')
fig.tight_layout()